In [1]:
import functools
import random
import textwrap

from huggingface_hub import InferenceApi

In [2]:
class CompletionError(Exception):
    """An error in attempting to complete text with BLOOM."""
    pass

In [3]:
class UnexpectedResponseError(AssertionError):
    """An unexpected error completing text. Probably a bug."""
    pass

In [4]:
class Supplier:
    """As the value of a BLOOM parameter, is called instead of used."""

    __slots__ = ('_func',)

    __match_args__ = ('_func',)

    def __init__(self, func):
        self._func = func
    
    def __repr__(self):
        return f'{type(self).__name__}({self._func!r})'
    
    def __call__(self):
        return self._func()

In [5]:
@functools.cache
def _get_inference_api():
    with open('.hf_token', encoding='utf-8') as file:
        api_token = file.read().strip()

    return InferenceApi(repo_id='bigscience/bloom', token=api_token)

In [6]:
class Completer:
    """High-level interface to Bloom completion."""

    __slots__ = ('_inference', '_text', '__dict__')

    def __init__(self, prompt, **parameters):
        """Create a completer for the given prompt and optional parameters."""
        self._inference = _get_inference_api()
        self.text = prompt

        if any(name.startswith('_') for name in parameters):
            raise TypeError('cannot set parameter name with leading "_"')
        
        self._set_defaults()
        self.__dict__.update(parameters)

    def __str__(self):
        """Prettified version of the text stored so far."""
        raw = self.text.strip().split('\n')
        pretty = ('\n'.join(textwrap.wrap(graf)) for graf in raw)
        return '\n\n'.join(pretty)
    
    @property
    def text(self):
        """The prompt plus any appended completions."""
        return self._text
    
    @text.setter
    def text(self, value):
        if not isinstance(value, str):
            raise TypeError('prompt must be a string')
        if not value:
            raise ValueError('prompt must be nonempty')
        self._text = value

    def __call__(self):
        """Attempt to complete the stored text, and print all text so far."""
        self.complete()
        print(self)
    
    def complete(self):
        """Attempt to complete the stored text, but do not print anything."""
        match self._inference(inputs=self.text, params=self._build_params()):
            case [{'generated_text': completion}]:
                self._text = completion  # Whatever BLOOM returns, we accept.
            case {'error': [*errors]}:
                raise CompletionError(*errors)
            case other_response:
                raise UnexpectedResponseError(other_response)
    
    def _set_defaults(self):
        self.do_sample = True
        self.max_new_tokens = 250
        self.seed = Supplier(random.random)
        self.temperature = 0.75
    
    def _build_params(self):
        return {name: (value() if isinstance(value, Supplier) else value)
                for name, value in sorted(self.__dict__.items())}


In [29]:
bloom1 = Completer("""
The color enforcers came around again. They ripped the cheese off our freshly
baked pizza. The customers dropped their food and ran out.

"Aha!" said the biggest, scariest henchman. "This sauce is red! We license the
color red to you, but you haven't been paying the license fees." I quaked in
fear. My grandfather had gotten both his knees busted by color enforcers.
""".strip().replace('\n', ' ').replace('  ', '\n'))

In [30]:
bloom1()

The color enforcers came around again. They ripped the cheese off our
freshly baked pizza. The customers dropped their food and ran out.

"Aha!" said the biggest, scariest henchman. "This sauce is red! We
license the color red to you, but you haven't been paying the license
fees." I quaked in fear. My grandfather had gotten both his knees
busted by color enforcers. The henchman waved a large piece of paper
(I figured it was a license). "We can give you a head start, but
you'll have to pay off this outstanding debt."

"No problem!" said my grandfather. He whipped out his wallet. "This
will get you started." He handed over a crisp, twenty-dollar bill.

"Really!" said the henchman. "I didn't realize what a generous company
you run. And that pizza was amazing!"

I was an executive of a prestigious software company in Seattle. I
worked on the Internet Explorer team, and we were in the middle of
improving web standards. We had heard about the standards and were
testing them.

"We heard about

In [9]:
bloom2 = Completer("""
Beverly understood how all the women in her family had fallen in love with this
energy ghost who lived in a lamp and fed parasitically off their vital energy
while making every carnal dream come to life. As a spacefaring physician
""".strip().replace('\n', ' '))

In [10]:
bloom2()

Beverly understood how all the women in her family had fallen in love
with this energy ghost who lived in a lamp and fed parasitically off
their vital energy while making every carnal dream come to life. As a
spacefaring physician, Beverly could tell that this energy ghost was
not even remotely healthy. It was far too focused on its own pleasure
and self-growth, and the people connected to it suffered. She could
also see that, by feeding off her vital energy, it was making its host
more sick than ever.

Beverly found she could not continue to focus on the patient as the
energy ghost started to close in on her. She could feel the ghost’s
presence binding her to her bed, and she was surprised by the sudden
surge of adrenaline that caused her to rage at it. She suddenly
started screaming, “You will not have this body!”

Beverly knew that the energy ghost could pick up on the most subtle of
emotions, but she also knew that the more energy she used to fight the
ghost, the more the ghost wou

In [21]:
# From https://aclanthology.org/2020.acl-main.463.pdf, Appendix A.
bloom3 = Completer("""
Robinson cried out, “Help! I’m being chased by a bear! All I have is these
sticks. What should I do?” The survivalist said, “Here’s what you can do.
""".strip().replace('\n', ' '))

In [22]:
bloom3()

Robinson cried out, “Help! I’m being chased by a bear! All I have is
these sticks. What should I do?” The survivalist said, “Here’s what
you can do.” “Throw one stick behind you. It’s a bear!” “Throw another
stick behind you. It’s a bear!” and “Throw another stick behind you.
It’s a bear!” Before they knew it, he was surrounded by hundreds of
other young men with sticks. He ran for his life! “Why did you do
that?” they yelled at him. “You told me to!” said the survivalist. Tim
Robinson had learned to follow orders without thinking.

Shows how easily we can be manipulated into doing what we would not
normally do! And what about the fake news media? Many people believe
everything they read, even though they know it’s not true! If we don’t
learn and train our brains to recognize, stop, and think before we
respond, we will be manipulated and bullied and controlled by others!

Shows how easily we can be manipulated into doing what we would not
normally do! And what about the fake news media